In [1]:
import time
from dask.distributed import Client

# Create a Dask client with 8 workers and 2 threads per worker
client = Client(n_workers=8, threads_per_worker=1, memory_limit='2GB')

# Print out the details of the Dask cluster
print(client)


<Client: 'tcp://127.0.0.1:38677' processes=8 threads=8, memory=14.90 GiB>


In [2]:
from google.cloud import storage
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator
from scipy.stats import chi2
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier, ExtraTreeRegressor, ExtraTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, ExtraTreesRegressor, ExtraTreesClassifier, AdaBoostRegressor
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

from scipy.stats.mstats import winsorize
from sklearn.decomposition import PCA
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings("ignore") # Suppress all warnings
pd.options.display.float_format = '{:.2f}'.format

In [3]:
# Initialize BigQuery and Storage clients
storage_client = storage.Client()

print("Authentication successful!")
storage_client = storage.Client()

Authentication successful!


In [4]:
import dask.dataframe as dd
import gcsfs



# Load CSV into Dask DataFrame
df = dd.read_csv(
    "gs://samplebucketvrs/Intelligence Analytics/og4/2019.csv",
    blocksize="50MB",
    assume_missing=True  # Converts int64 to float64 if missing values exist
)

In [5]:
# Check Data
#df.head()

In [6]:
import dask.dataframe as dd

# Describe dataset (computes statistics on a sample)
#print(df.describe().compute())

# Drop 'id' column if it exists
columns_to_drop = [col for col in df.columns if col.lower() == 'id']
if columns_to_drop:
    df = df.drop(columns=columns_to_drop)

# Convert 'Date' column to datetime using Dask
df['FL_DATE'] = df['FL_DATE'].map_partitions(pd.to_datetime, meta=('FL_DATE', 'datetime64[ns]'))

# Verify changes
dtypes = df.dtypes
#print(dtypes)

In [7]:
df = df.drop(columns=["Unnamed: 20"])

In [ ]:
import numpy as np
from scipy.stats.mstats import winsorize

def detect_outliers_dask(df, method="iqr", threshold=1.5):
    """
    Detect outliers in a large Dask DataFrame using IQR or Z-score.
    Returns a dictionary with column names and percentage of outliers.
    """
    outlier_percentages = {}

    for col in df.select_dtypes(include=[np.number]).columns:
        if method == "iqr":
            Q1, Q3 = df[col].quantile([0.25, 0.75]).compute(split_out=4)  
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
            mask = (df[col] < lower_bound) | (df[col] > upper_bound)
        elif method == "zscore":
            mean, std = df[col].mean().compute(split_out=4), df[col].std().compute(split_out=4)
            mask = abs((df[col] - mean) / std) > threshold
        else:
            raise ValueError("Method must be 'iqr' or 'zscore'.")

        outlier_count = mask.sum().compute()
        total_rows = len(df)
        outlier_percentages[col] = (outlier_count / total_rows) * 100
    
    return outlier_percentages

def clean_or_winsorize_dask(df, outlier_percentages, threshold=5):
    """
    Cleans or applies Winsorization based on outlier percentage.
    """
    for col, percentage in outlier_percentages.items():
        if percentage <= threshold:
            print(f"Removing {percentage:.2f}% outliers from column '{col}'.")
            Q1, Q3 = df[col].quantile([0.25, 0.75]).compute(split_out=4)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
            df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
        else:
            print(f"Applying Winsorization to column '{col}' with {percentage:.2f}% outliers.")
            df[col] = df[col].map_partitions(lambda x: winsorize(x, limits=(0.05, 0.05)), meta=(col, 'float64'))

    return df

# ✅ **Step 1: Detect Outliers in the Existing df**
outlier_percentages = detect_outliers_dask(df, method="iqr", threshold=1.5)

# ✅ **Step 2: Clean Data**
df_cleaned = clean_or_winsorize_dask(df, outlier_percentages, threshold=5)


In [ ]:
import dask.dataframe as dd
import pandas as pd
import numpy as np
from dask_ml.preprocessing import LabelEncoder
from dask import delayed
from sklearn.preprocessing import OneHotEncoder

# Identify numerical and categorical columns
numerical_cols = df.select_dtypes(include=["number"]).columns
categorical_cols = df.select_dtypes(include=["object", "category","string"]).columns

# ✅ Impute missing values for numerical columns using median
for col in numerical_cols:
    median_value = df[col].median_approximate().compute()  # Use approximate median for efficiency
    df[col] = df[col].fillna(median_value)
    print(f"Imputed median for {col}: {median_value}")

# ✅ Encode categorical variables
for col in categorical_cols:
    unique_count = df[col].nunique().compute()  # Get number of unique values
    
    if unique_count <= 10:
        # **One-Hot Encoding** for low-cardinality categorical columns
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
        
        # Apply One-Hot Encoding using map_partitions
        df = df.map_partitions(
            lambda d: d.join(
                pd.DataFrame(
                    encoder.fit_transform(d[[col]]),
                    index=d.index,
                    columns=[f"{col}_{i}" for i in range(unique_count)]
                )
            ), 
            meta=df
        )
        
        df = df.drop(columns=[col])  # Drop original column after encoding
        print(f"Applied One-Hot Encoding to {col}")

    else:
        # **Label Encoding** for high-cardinality categorical columns
        encoder = LabelEncoder()
        df[col] = df[col].map_partitions(lambda x: encoder.fit_transform(x), meta=(col, 'int64'))
        print(f"Applied Label Encoding to {col}")

# ✅ Check the transformed DataFrame
print(df.dtypes)

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [ ]:
import dask.dataframe as dd
import pandas as pd
import dask.array as da
from dask_ml.preprocessing import StandardScaler
from dask_ml.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, ExtraTreesClassifier
import dask.delayed

# ✅ Sample 10% of the data
sampled_df = df.dropna(subset=["DEP_DELAY"]).sample(frac=0.1, random_state=42)

# ✅ Separate features (X) and target (y)
target_col = "DEP_DELAY"
y = sampled_df[target_col]
X = sampled_df.drop(columns=[target_col, "FL_DATE"])



# ✅ Standardize numerical features with Dask's StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ✅ Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# ✅ Compute Dask DataFrame to NumPy (for model compatibility)
X_train_np, X_test_np, y_train_np, y_test_np = [
    arr.compute() for arr in [X_train, X_test, y_train, y_test]
]

# ✅ Ensure task type and model selection
if y_train_np.nunique() <= 10:
    model = ExtraTreesClassifier(random_state=42, n_jobs=-1, n_estimators=100)
    task_type = "Classification"
else:
    model = ExtraTreesRegressor(random_state=42, n_jobs=-1, n_estimators=100)
    task_type = "Regression"

# ✅ Train the model
model.fit(X_train_np, y_train_np)

print(f"✅ {task_type} Model Training Complete!")
